In [1]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import glob
import numpy as np
from moviepy.editor import VideoFileClip
%matplotlib inline

/Users/Rayno/anaconda3/lib/python3.5/site-packages/skimage/filter/__init__.py:6: skimage_deprecation: The `skimage.filter` module has been renamed to `skimage.filters`.  This placeholder module will be removed in v0.13.
  warn(skimage_deprecation('The `skimage.filter` module has been renamed '


In [2]:
def showImage(image):
    plt.imshow(image); plt.axis('off'); plt.show();
    
def showImages(images):
    num_y = int(len(images)/3) + 1
    fig = plt.figure(figsize = (10, len(images)*2))
    grid = gridspec.GridSpec(num_y, 3, wspace = 0.1, hspace = 0.1, width_ratios=np.ones(shape=(3)), height_ratios=np.ones(shape=(len(images))))
    for i, image in enumerate(images):
        subp = plt.subplot(grid[i])
        subp.imshow(image)
        plt.axis('off')
        fig.add_subplot(subp)
    plt.show();

def renderImages(images, k=3):
    num_y = int(len(images)/k) + 1
    fig = plt.figure(figsize = (3, len(images)/2))
    grid = gridspec.GridSpec(num_y, k, wspace = 0., hspace = 0., width_ratios=np.ones(shape=(k)), height_ratios=np.ones(shape=(num_y)))
    for i, image in enumerate(images):
        subp = plt.subplot(grid[i])
        subp.imshow(image)
        plt.axis('off')
        fig.add_subplot(subp)
    
    plt.savefig ( "./latest.png" )
    return(cv2.imread("./latest.png"))

# Camera Calibration

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image. Thus, objp is just a replicated array of coordinates, and objpoints will be appended with a copy of it every time I successfully detect all chessboard corners in a test image. imgpoints will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.

In [3]:
dims = (9,6)
calibration_images = [mpimg.imread(file) for file in glob.glob("./camera_cal/calibration*.jpg")]
grey_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in calibration_images]
chessboard_corners = [cv2.findChessboardCorners(image, dims, None) for image in grey_images]

In [4]:
images_with_corners = [cv2.drawChessboardCorners(calibration_images[i], dims, *chessboard_corners[i][::-1]) for i in range(len(calibration_images))]

In [5]:
objp = np.zeros((np.prod(dims), 3), np.float32)
objp[:,:2] = np.mgrid[0:dims[0], 0:dims[1]].T.reshape(-1,2)

objpoints = [objp for corner_set in chessboard_corners if corner_set[0]]
imgpoints = [corner_set[1] for corner_set in chessboard_corners if corner_set[0]]

I then used the output objpoints and imgpoints to compute the camera calibration and distortion coefficients using the cv2.calibrateCamera() function. I applied this distortion correction to the test image using the cv2.undistort() function and obtained this result:

In [6]:
# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, grey_images[0].shape[::-1], None, None)

In [7]:
# Undistort the images
undistorted_images = [cv2.undistort(image, mtx, dist, None, mtx) for image in calibration_images]

# Lane Line Detection

In [8]:
# Define Perspective Transform function
def warp(img, offset = 0.2, bot_width = 0.76, mid_width = 0.08, height_pct = 0.62, bottom_trim = 0.935):
    w = img.shape[1]
    h = img.shape[0]
    
    src = np.float32([[w*(0.5 - mid_width/2), h*height_pct],
                      [w*(0.5 + mid_width/2), h*height_pct], 
                      [w*(0.5 + bot_width/2), h*bottom_trim], 
                      [w*(0.5-bot_width/2),   h*bottom_trim]])
    
    #offset = .25*w
    offset = .2*w
    w_offset = .2*w
    dst = np.float32([ [offset,   0], 
                       [w-offset*1.2, 0],
                       [w-offset*1.2, h], 
                       [offset,   h]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)
    
    warped = cv2.warpPerspective(img, M, (w, h), flags = cv2.INTER_LINEAR)
    return warped, M, M_inv

# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=20, thresh_max=100):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

def color_threshold(image, hls_sthresh=(0,255), hsv_sthresh=(0,255), hsv_vthresh=(0,255)):
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    v_channel = hsv[:,:,2]
    v_binary = np.zeros_like(v_channel)
    v_binary[(v_channel >= hsv_vthresh[0]) & (v_channel <= hsv_vthresh[1])] = 1
    
    #hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s_channel = hsv[:,:,1]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > hsv_sthresh[0]) & (s_channel <= hsv_sthresh[1])] = 1
    
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    s2_channel = hls[:,:,2]
    s2_binary = np.zeros_like(s2_channel)
    s2_binary[(s2_channel > hls_sthresh[0]) & (s2_channel <= hls_sthresh[1])] = 1
    
    """luv = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
    s2_channel = hls[:,:,2]
    s2_binary = np.zeros_like(s2_channel)
    s2_binary[(s2_channel > hls_sthresh[0]) & (s2_channel <= hls_sthresh[1])] = 1"""
     
    output = np.zeros_like(s_channel)
    output[(s_binary == 1) & (v_binary == 1) & (s2_binary == 1)] = 1
    
    return output



In [9]:
"""I detected lane lines from the warped image using a sliding-window-based tracking method 
(credit: Tutorial Video on P4 by Udacity). This involves firstly summing the vertical pixel
values in the image and identifying the 2 peaks in the histogram, which should give an 
indication of the location of the lane lines.

We identify the centers of these peaks and start at the corresponding point in the warped 
image. We then move up and down, and left and right within certain acceptable limits, 
identifying the next possible set of pixels that might form part of the lane line.
This is done in the method below (credit: P4 Tutorial Video, Udacity)
"""


class tracker():
    def __init__(self, Mywindow_width, Mywindow_height, Mymargin, Mysmooth_factor = 15):
        
        self.recent_centers = []
        self.window_width = Mywindow_width
        self.window_height = Mywindow_height
        self.margin = Mymargin
        self.smooth_factor = Mysmooth_factor
        
    def find_window_centroids(self, warped):
        
        window_width = self.window_width
        window_height = self.window_height
        margin = self.margin
        
        window_centroids = []
        window = np.ones(window_width)
        
        l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window, l_sum)) - window_width/2
        r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window, r_sum)) - window_width/2 + int(warped.shape[1]/2)
        
        window_centroids.append((l_center, r_center))
        
        for level in range(1, (int)(warped.shape[0]/window_height)):
            image_layer = np.sum(warped[int(warped.shape[0] - (level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            
            offset = window_width/2
            l_min_index = int(max(l_center + offset - margin, 0))
            l_max_index = int(min(l_center + offset + margin, warped.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index]) + l_min_index - offset
            
            r_min_index = int(max(r_center + offset - margin, 0))
            r_max_index = int(min(r_center + offset + margin, warped.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index: r_max_index]) + r_min_index - offset
            
            window_centroids.append((l_center, r_center))
            
        self.recent_centers.append(window_centroids)
        
        return np.average(self.recent_centers[-self.smooth_factor:], axis = 0)
    
    def detect_lines(self, warped):
        
        window_centroids = self.find_window_centroids (warped)

        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)

        leftx = []
        rightx = []

        for level in range(0, len(window_centroids)):

            leftx.append(window_centroids[level][0])
            rightx.append(window_centroids[level][1])

            l_mask = window_mask(self.window_width, self.window_height, warped, window_centroids[level][0], level)
            r_mask = window_mask(self.window_width, self.window_height, warped, window_centroids[level][1], level)

            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        template = np.array(r_points + l_points, np.uint8)
        zero_channel = np.zeros_like(template)
        template = np.array(cv2.merge((zero_channel, template, zero_channel)), np.uint8)
        warpage = np.array(cv2.merge((warped, warped, warped)), np.uint8)
        detected_lanes = cv2.addWeighted(warpage, 1, template, 0.5, 0.0)
        
        return detected_lanes, leftx, rightx
        
def window_mask(width, height, img_ref, center, level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0] - (level + 1)*height):int(img_ref.shape[0] - level*height), max(0, int(center-width)):min(int(center+width), img_ref.shape[1])] = 1
    return output

In [10]:
class lane_fitter():
    def __init__(self):
        self.previous_left_MSE = float('inf')
        self.previous_right_MSE = float('inf')
        self.previous_left_poly = np.zeros(shape=(3))
        self.previous_right_poly = np.zeros(shape=(3))
        self.previous_7_left_lanes = []
        self.previous_7_right_lanes = []
        
    def fit_lanes(self, current_image, window_width, window_height, leftx, rightx):
        if len(self.previous_7_left_lanes) > 9:
            self.previous_7_left_lanes = self.previous_7_left_lanes[-9:]
        if len(self.previous_7_right_lanes) > 9:
            self.previous_7_right_lanes = self.previous_7_right_lanes[-9:]
            
        #Next, I fit a polynomial to the curve:
            
        yvals = range(0, current_image.shape[0])
        res_yvals = np.arange(current_image.shape[0] - (window_height/2), 0, -window_height)

        left_fit, left_res, _, _, _ = np.polyfit(res_yvals, leftx, 2, full=True)
        right_fit, right_res, _, _, _ = np.polyfit(res_yvals, rightx, 2, full=True)
        
        self.previous_7_left_lanes.append(left_fit)
        self.previous_7_right_lanes.append(right_fit)
        
        left_fit = np.average(self.previous_7_left_lanes, axis = 0)
        right_fit = np.average(self.previous_7_right_lanes, axis = 0)
        
        #I used the MSE of the fit to inform a weighted average across successive frames.
        """
        MSE = np.mean(np.square(left_res[1:] - right_res[1:]))
        
        #Bayesian informing
        k = MSE/(MSE + self.previous_left_MSE)
        left_fit = k*self.previous_left_poly + (1-k)*left_fit

        k = MSE/(MSE + self.previous_right_MSE)
        right_fit = k*self.previous_right_poly + (1-k)*right_fit

        if self.previous_right_MSE < float('inf'):
            self.previous_right_MSE = k*self.previous_right_MSE + (1-k)*MSE
        else:
            self.previous_right_MSE = MSE   
        self.previous_right_poly = right_fit

        if self.previous_left_MSE < float('inf'):
            self.previous_left_MSE = k*self.previous_left_MSE + (1-k)*MSE
        else:
            self.previous_left_MSE = MSE   
        self.previous_left_poly = left_fit"""

        left_fitx = left_fit[0]*yvals*yvals + left_fit[1]*yvals + left_fit[2]
        left_fitx = np.array(left_fitx, np.int32)

        right_fitx = right_fit[0]*yvals*yvals + right_fit[1]*yvals + right_fit[2]
        right_fitx = np.array(right_fitx, np.int32)

        left_lane = np.array(list(zip(np.concatenate((left_fitx - window_width/2, left_fitx[::-1] + window_width/2), axis = 0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)
        right_lane = np.array(list(zip(np.concatenate((right_fitx - window_width/2, right_fitx[::-1] + window_width/2), axis = 0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)
        inner_lane = np.array(list(zip(np.concatenate((left_fitx + window_width/2, right_fitx[::-1] - window_width/2), axis=0), np.concatenate((yvals, yvals[::-1]), axis = 0))), np.int32)
        
        #Assuming the lane is about 30 meters long and 3.7 meters wide:
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/720 # meters per pixel in x dimension

        # Firstly I fit a second order polynomial (curve) to the left lane. I then used this fitted curve to calculate the angle of the left lane line's curvature. 
        
        corner_curve = np.polyfit(x = np.array(res_yvals, np.float32)*ym_per_pix, 
                                  y = np.array(leftx, np.float32)*xm_per_pix, 
                                  deg = 2)
        #curve_radians = ((1 + (2*corner_curve[0]*np.max(yvals[-1]*ym_per_pix + corner_curve[1])**2)**1.5) / np.absolute(2*corner_curve[0])
        curve_radians = ((1 + (2*corner_curve[0]*np.max(yvals)*ym_per_pix + corner_curve[1])**2)**1.5) / np.absolute(2*corner_curve[0])

        camera_center = (left_fitx[-1] + right_fitx[-1])/2
        center_diff = (camera_center-current_image.shape[1]/2)*xm_per_pix
        
        side_pos = 'left'
        if center_diff <= 0:
            side_pos = "right"
    
        return left_lane, inner_lane, right_lane, curve_radians, center_diff, side_pos

In [11]:
def draw_road_lanes(image, left_lane, inner_lane, right_lane, M_inv):
    road = np.zeros_like(image)
    road_bg = np.zeros_like(image)
    
    cv2.fillPoly(road, [left_lane], color=[255,0,0])
    cv2.fillPoly(road, [right_lane], color=[0,0,255])
    cv2.fillPoly(road, [inner_lane], color=[0,0,255])
    cv2.fillPoly(road_bg, [left_lane], color = [255, 255, 255])
    cv2.fillPoly(road_bg, [right_lane], color = [255, 255, 255])
    
    road_warped = cv2.warpPerspective(road, M_inv, (image.shape[1], image.shape[0]), flags = cv2.INTER_LINEAR)
    road_warped_bg = cv2.warpPerspective(road_bg, M_inv, (image.shape[1], image.shape[0]), flags = cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(image, 1.0, road_warped_bg, -1.0, 0.0)
    result = cv2.addWeighted(base, 1.0, road_warped, 1.0, 0.0)
    
    return result

#I used a combination of color and gradient thresholds to generate a binary image"


def process_LUV(image):
    luv = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
    yellow_mask = cv2.inRange(luv, np.array([140,110,120]), np.array([230, 130, 255]))
    yellow_lane = cv2.bitwise_and(luv,luv, mask= yellow_mask)
    
    white_mask = cv2.inRange(luv, np.array([200,0,100]), np.array([255, 130, 220]))
    color_mask = cv2.bitwise_or(white_mask, yellow_mask)
    return color_mask

"""hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    hls_l = hls[:,:,1]
    gradx_mask = abs_sobel_thresh(hls_l, orient='x', thresh_max=225, thresh_min=50)
    grady_mask = abs_sobel_thresh(hls_l, orient='y', thresh_max=225, thresh_min=50)
    grad_mask_1 = np.copy(cv2.bitwise_or(gradx_mask, grady_mask))
    
    hls_s = hls[:,:,2]
    gradx_mask = abs_sobel_thresh(hls_s, orient='x', thresh_max=225, thresh_min=50)
    grady_mask = abs_sobel_thresh(hls_s, orient='y', thresh_max=225, thresh_min=50)
    grad_mask_2 = np.copy(cv2.bitwise_or(gradx_mask, grady_mask))
    
    sobel = cv2.GaussianBlur(cv2.bitwise_or(grad_mask_1, grad_mask_2), (25, 25), 0)
    
    white_color_mask = cv2.inRange(hsv, (20, 0, 180), (255, 80, 255))
    yellow_color_mask = cv2.inRange(hsv, (0, 100, 100), (50, 255, 255))
    c_binary = cv2.bitwise_or(yellow_color_mask, white_color_mask)"""

def apply_masks(img):
    
    color_mask = process_LUV(img)
    
    preprocessedImage = np.zeros_like(img[:,:,0])

    gradx_mask = abs_sobel_thresh(img, orient='x', thresh_max=225, thresh_min=25)
    grady_mask = abs_sobel_thresh(img, orient='y', thresh_max=225, thresh_min=50)

    mag_th = mag_thresh(img, sobel_kernel=9, mag_thresh=(50, 250))
    dir_th = dir_threshold(img, sobel_kernel=15, thresh=(0.7, 1.3))

    white_color_mask = color_threshold(img, hls_sthresh = (88, 255), hsv_sthresh=(0, 70), hsv_vthresh=(160, 255))
    yellow_color_mask = color_threshold(img, hls_sthresh = (88, 190), hsv_vthresh=(100, 255))

    c_binary = cv2.bitwise_or(yellow_color_mask, white_color_mask)

    preprocessedImage[((gradx_mask == 1) & (grady_mask == 1))  | (color_mask==255) | ((yellow_color_mask == 1) | (white_color_mask == 1)) & (mag_th == 1) & (dir_th == 1)] = 255
    
    return preprocessedImage

In [17]:
current_lane_fitter = lane_fitter()

def reset_lane_fitter():
    global current_lane_fitter
    current_lane_fitter = lane_fitter()
        
def map_lane(img):
    global lane_fitter
    img = cv2.undistort(img, mtx, dist, None, mtx)
    
    preprocessedImage = apply_masks(img)
    
    warped, M, M_inv = warp(preprocessedImage, bot_width =0.6, height_pct=0.629)

    window_width = 35
    window_height = 80
    curve_centers = tracker(Mywindow_width = 45, Mywindow_height = 80, Mymargin = 40, Mysmooth_factor = 100)

    detected_lanes, leftx, rightx = curve_centers.detect_lines(warped)

    left_lane, inner_lane, right_lane, curve_radians, center_diff, side_pos = current_lane_fitter.fit_lanes(warped, window_width, window_height, leftx, rightx)

    result = draw_road_lanes(img, left_lane, inner_lane, right_lane, M_inv)

    cv2.putText(result, 'Radius of Curvature: ' + str(round(curve_radians, 3)) + '(m)', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, str(abs(round(center_diff, 3))) + 'm ' + side_pos + ' of the center', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    return preprocessedImage, warped, detected_lanes, result

def process_4_images(image):
    preprocessImage, warped, detected_lanes, result = map_lane(image)
    #preprocessImage = map_lane(image)
    vis_left = np.concatenate((cv2.cvtColor(preprocessImage,cv2.COLOR_GRAY2RGB), detected_lanes), axis=0)
    vis_right = np.concatenate((cv2.cvtColor(warped,cv2.COLOR_GRAY2RGB), result), axis=0)
    vis = np.concatenate((vis_left, vis_right), axis=1)
    return vis

def process_result(image):
    preprocessImage, warped, detected_lanes, result = map_lane(image)
    #preprocessImage = map_lane(image)
    
    return result

In [19]:
Output_video = 'project_video_output'
Input_video = 'project_video.mp4'

reset_lane_fitter()

clip1 =  VideoFileClip(Input_video)
video_clip = clip1.fl_image(process_result)
video_clip.write_videofile(Output_video + ".mp4", audio = False)


reset_lane_fitter()

clip2 =  VideoFileClip(Input_video)#.subclip(20, 30)
video_clip2 = clip2.fl_image(process_4_images)
video_clip2.write_videofile(Output_video + "_show_pipeline.mp4", audio = False)

/Users/Rayno/anaconda3/lib/python3.5/site-packages/moviepy/audio/io/readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))



MoviePy: building video file project_video_output.mp4
----------------------------------------

Writing video into project_video_output.mp4


/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:56: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future




  0%|          | 0/1210 [00:00<?, ?it/s]/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:56: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future




  0%|          | 1/1210 [00:00<08:10,  2.47it/s]



  0%|          | 2/1210 [00:00<08:21,  2.41it/s]



  0%|          | 3/1210 [00:01<08:20,  2.41it/s]



  0%|          | 4/1210 [00:01<0

Done writing video in project_video_output.mp4 !
Your video is ready !


/Users/Rayno/anaconda3/lib/python3.5/site-packages/moviepy/audio/io/readers.py:110: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  self.nchannels))



MoviePy: building video file project_video_output_show_pipeline.mp4
----------------------------------------

Writing video into project_video_output_show_pipeline.mp4


/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:56: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future




  0%|          | 0/1210 [00:00<?, ?it/s]/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:53: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Rayno/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:56: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future




  0%|          | 1/1210 [00:01<23:17,  1.16s/it]



  0%|          | 2/1210 [00:02<23:47,  1.18s/it]



  0%|          | 3/1210 [00:03<23:08,  1.15s/it]



  0%|          | 4/1210 [00:04<2

Done writing video in project_video_output_show_pipeline.mp4 !
Your video is ready !
